In [1]:
#export
import k1lib as _k1lib, torch.nn as _nn, torch as _torch, os as _os
import dill as _dill

In [2]:
#export
class Learner:
    def __init__(self, model:_nn.Module, *args, **kwargs):
        """Initializes a Learner. Intended to have an outside factory function to create this and not be used directly"""
        self.model = model; self.data = None; self.opt = None; self.lossF = None
        self.configure(*args, **kwargs)
        self.withCbs(_k1lib.Callbacks.standard())
        self.fileName = None
    def configure(self, data:_k1lib.Data=None, opt=None, lossF=None):
        self.data = self.data if data is None else data
        self.opt = opt or self.opt
        self.lossF = lossF or self.lossF
        return self
    def withCbs(self, cbs): self.cbs = cbs; cbs.injectLearner(self); return self
    def _run1Batch(self):
        self.cbs("startBatch")
        try:
            self.y = self.model(self.xb); self.cbs("endPass")
            loss = self.lossF(self.y, self.yb); self.loss = loss.detach().item(); self.cbs("endLoss")
            if not self.cbs("startBackward"): loss.backward()
            if not self.cbs("startStep"):  self.opt.step()
            if not self.cbs("startZeroGrad"): self.opt.zero_grad()
        except _k1lib.CancelBatchException: self.cbs("cancelBatch")
        self.cbs("endBatch")
    def _run1Epoch(self):
        self.cbs("startEpoch")
        try:
            self.nBatches = len(self.data.train); self.model.train()
            for self.batch, (self.xb, self.yb) in enumerate(self.data.train):
                self._run1Batch()
            self.nBatches = len(self.data.valid); self.model.eval(); self.cbs("startValidBatches")
            for self.batch, (self.xb, self.yb) in enumerate(self.data.valid):
                self._run1Batch()
        except _k1lib.CancelEpochException: self.cbs("cancelEpoch")
        self.cbs("endEpoch")
    def run(self, epochs, *args, **kwargs):
        """Runs the model for a specified number of epochs."""
        self.epochs = epochs; self.configure(None, *args, **kwargs)
        if self.data == None: raise Exception("No data object found")
        if self.opt == None: raise Exception("No optimizer found")
        if self.lossF == None: raise Exception("No loss function found")
        self.cbs("startRun")
        try:
            for self.epoch in range(epochs):
                self._run1Epoch()
        except _k1lib.CancelRunException: self.cbs("cancelRun")
        self.cbs("endRun"); return self
    def save(self, fileName=None):
        self.fileName = fileName or self.fileName
        if self.fileName == None:
            files = [file for file in _os.listdir() if file.startswith("learner") and file.endswith(".pth")]
            files = set([int(file.split(".pth")[0].split("learner-")[1]) for file in files])
            count = 0;
            while count in files: count += 1
            self.fileName = f"learner-{count}.pth"
        _torch.save(self, self.fileName, pickle_module=_dill)
        print(f"Saved to {self.fileName}")
    def __getattr__(self, attr):
        if attr == "cbs": raise AttributeError()
        if hasattr(self.cbs, attr): return getattr(self.cbs, attr)
        raise AttributeError()
    def __getstate__(self): return self.__dict__
    def __setstate__(self, state):
        self.__dict__.update(state); self.cbs.injectLearner(self)
    @staticmethod
    def load(fileName=None):
        if fileName==None: fileName = input("Enter learner file name to load:")
        print(f"Loaded from {fileName}"); return _torch.load(fileName, pickle_module=_dill)
    def __repr__(self):
        return f"""l.model:\n{_k1lib.tab(str(self.model))}
l.opt:\n{_k1lib.tab(str(self.opt))}
l.cbs:\n{_k1lib.tab(self.cbs.__repr__())}
Use...
- l.configure(...): to change any configs (data, model, opt, lossF)
- l.withCbs(...): to use a custom `Callbacks` object
- l.run(epochs): to run the network
- l.Loss: to get a specific callback\n\n"""

In [3]:
!../export.py _learner

Current dir: /home/kelvin/repos/labs/k1lib, ../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> build/lib/k1lib
copying k1lib/viz.py -> build/l

In [33]:
import torch

In [28]:
l = Lambda(lambda x: x[None])

In [32]:
l(torch.rand(2, 3))

tensor([[[0.0042, 0.3004, 0.6085],
         [0.9042, 0.8013, 0.1034]]])

In [10]:
a = {"a": 3, "b": 5}
a.update({"b": 8, "c": 11})

In [11]:
a

{'a': 3, 'b': 8, 'c': 11}